In [1]:
import pandas as pd
import glob
import numpy as np
from datetime import timedelta


In [2]:
data_path = "D:/unified_metor_content/fraud_detection/fraud_detection/" 

file_list = sorted(glob.glob(data_path + "*.pkl"))

# Read and concatenate all daily files
df = pd.concat([pd.read_pickle(file) for file in file_list], ignore_index=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754155 entries, 0 to 1754154
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   TRANSACTION_ID     int64         
 1   TX_DATETIME        datetime64[ns]
 2   CUSTOMER_ID        object        
 3   TERMINAL_ID        object        
 4   TX_AMOUNT          float64       
 5   TX_TIME_SECONDS    object        
 6   TX_TIME_DAYS       object        
 7   TX_FRAUD           int64         
 8   TX_FRAUD_SCENARIO  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 120.4+ MB


In [4]:
sum_ter = df[df['TX_FRAUD'] == 1].groupby('TERMINAL_ID')['TX_FRAUD'].count()

In [5]:
df['sum_on_terminal'] = df['TERMINAL_ID'].map(sum_ter).fillna(0)

In [6]:
df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,sum_on_terminal
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,17.0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,1.0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,1.0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,0.0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-30 23:56:36,161,655,54.24,15810996,182,0,0,0.0
1754151,1754151,2018-09-30 23:57:38,4342,6181,1.23,15811058,182,0,0,2.0
1754152,1754152,2018-09-30 23:58:21,618,1502,6.62,15811101,182,0,0,0.0
1754153,1754153,2018-09-30 23:59:52,4056,3067,55.40,15811192,182,0,0,0.0


In [7]:
df.sort_values(['TERMINAL_ID', 'TX_DATETIME'])

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,sum_on_terminal
9579,9579,2018-04-02 01:00:01,3440,0,16.07,90001,1,0,0,0.0
12806,12806,2018-04-02 09:49:55,3302,0,67.83,121795,1,0,0,0.0
24184,24184,2018-04-03 12:14:41,3790,0,26.82,216881,2,0,0,0.0
46284,46284,2018-04-05 16:47:41,1125,0,40.45,406061,4,0,0,0.0
58807,58807,2018-04-07 06:05:21,1125,0,48.39,540321,6,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
1732586,1732586,2018-09-28 15:34:24,4313,9999,26.86,15608064,180,0,0,1.0
1745583,1745583,2018-09-30 05:36:08,53,9999,4.84,15744968,182,0,0,1.0
1747229,1747229,2018-09-30 09:00:10,1661,9999,5.88,15757210,182,0,0,1.0
1752796,1752796,2018-09-30 17:47:37,4961,9999,97.12,15788857,182,0,0,1.0


In [8]:
# Custom aggregation: Rolling standard deviation
df['Rolling_terminal_count'] = (
    df.groupby('TERMINAL_ID')
            .rolling('28D', on='TX_DATETIME')['TX_FRAUD']
            .count()
            .reset_index(drop=True)
)
print(df)


         TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  \
0                     0 2018-04-01 00:00:31         596        3156   
1                     1 2018-04-01 00:02:10        4961        3412   
2                     2 2018-04-01 00:07:56           2        1365   
3                     3 2018-04-01 00:09:29        4128        8737   
4                     4 2018-04-01 00:10:34         927        9906   
...                 ...                 ...         ...         ...   
1754150         1754150 2018-09-30 23:56:36         161         655   
1754151         1754151 2018-09-30 23:57:38        4342        6181   
1754152         1754152 2018-09-30 23:58:21         618        1502   
1754153         1754153 2018-09-30 23:59:52        4056        3067   
1754154         1754154 2018-09-30 23:59:57        3542        9849   

         TX_AMOUNT TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  \
0            57.16              31            0         0          

In [9]:
df['Rolling_terminal_std'] = (
    df.groupby('TERMINAL_ID')
            .rolling('28D', on='TX_DATETIME')['Rolling_terminal_count']
            .std()
            .reset_index(drop=True)
)

In [10]:
df['Rolling_customer_amt'] = (
    df.groupby('CUSTOMER_ID')
            .rolling('14D', on='TX_DATETIME')['TX_AMOUNT']
            .sum()
            .reset_index(drop=True)
)
print(df)

         TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  \
0                     0 2018-04-01 00:00:31         596        3156   
1                     1 2018-04-01 00:02:10        4961        3412   
2                     2 2018-04-01 00:07:56           2        1365   
3                     3 2018-04-01 00:09:29        4128        8737   
4                     4 2018-04-01 00:10:34         927        9906   
...                 ...                 ...         ...         ...   
1754150         1754150 2018-09-30 23:56:36         161         655   
1754151         1754151 2018-09-30 23:57:38        4342        6181   
1754152         1754152 2018-09-30 23:58:21         618        1502   
1754153         1754153 2018-09-30 23:59:52        4056        3067   
1754154         1754154 2018-09-30 23:59:57        3542        9849   

         TX_AMOUNT TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  \
0            57.16              31            0         0          

In [11]:
df['Rolling_customer_std'] = (
    df.groupby('CUSTOMER_ID')
            .rolling('14D', on='TX_DATETIME')['Rolling_customer_amt']
            .std()
            .reset_index(drop=True)
)

In [12]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [13]:
df = df.drop(columns=['TX_DATETIME'])


In [14]:
df['TX_TIME_SECONDS'] = pd.to_numeric(df['TX_TIME_SECONDS'], errors='coerce')
df['TX_TIME_DAYS'] = pd.to_numeric(df['TX_TIME_DAYS'], errors='coerce')


In [15]:
from sklearn.preprocessing import LabelEncoder

for col in ['CUSTOMER_ID', 'TERMINAL_ID']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754155 entries, 0 to 1754154
Data columns (total 13 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   TRANSACTION_ID          int64  
 1   CUSTOMER_ID             int32  
 2   TERMINAL_ID             int32  
 3   TX_AMOUNT               float64
 4   TX_TIME_SECONDS         int64  
 5   TX_TIME_DAYS            int64  
 6   TX_FRAUD                int64  
 7   TX_FRAUD_SCENARIO       int64  
 8   sum_on_terminal         float64
 9   Rolling_terminal_count  float64
 10  Rolling_terminal_std    float64
 11  Rolling_customer_amt    float64
 12  Rolling_customer_std    float64
dtypes: float64(6), int32(2), int64(5)
memory usage: 160.6 MB


In [17]:
from sklearn.model_selection import train_test_split

# Separate features & target
X = df.drop(columns=['TX_FRAUD','TX_FRAUD_SCENARIO'])
y = df['TX_FRAUD']

# Split BEFORE any balancing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [18]:
from sklearn.utils import resample
import pandas as pd

# Combine back for easier resampling
train_df = pd.concat([X_train, y_train], axis=1)

# Split into majority/minority
df_majority = train_df[train_df.TX_FRAUD == 0]
df_minority = train_df[train_df.TX_FRAUD == 1]

# Downsample majority to 5x minority
df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority)*5,  # 5:1 ratio
    random_state=42
)

# Combine and shuffle
df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and labels again
X_train_bal = df_balanced.drop(columns=['TX_FRAUD'])
y_train_bal = df_balanced['TX_FRAUD']


In [19]:
#pip install lightgbm


In [20]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Train on balanced data (either SMOTE or downsampled)
model = LGBMClassifier(random_state=42)
model.fit(X_train_bal, y_train_bal)

# Predict on original test set (imbalanced)
y_pred = model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Number of positive: 11745, number of negative: 58725
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2344
[LightGBM] [Info] Number of data points in the train set: 70470, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[[337603  10292]
 [   389   2547]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    347895
           1       0.20      0.87      0.32      2936

    accuracy                           0.97    350831
   macro avg       0.60      0.92      0.65    350831
weighted avg       0.99      0.97      0.98    350831



In [21]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42, scale_pos_weight=5)  # scale_pos_weight helps imbalance
model.fit(X_train_bal, y_train_bal)
# Predict on original test set (imbalanced)
y_pred = model.predict(X_test)

# Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[332209  15686]
 [   266   2670]]
              precision    recall  f1-score   support

           0       1.00      0.95      0.98    347895
           1       0.15      0.91      0.25      2936

    accuracy                           0.95    350831
   macro avg       0.57      0.93      0.61    350831
weighted avg       0.99      0.95      0.97    350831

